In [1]:
# use the GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [7]:
%pip install bitsandbytes


     -------------------------------------- 101.6/101.6 MB 6.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: C:\Users\nik_b\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")  #  load_in_8bit=True Quantization, not working at the moment, for some reason...
model.to(device)
print("done")

done


In [21]:
from transformers import AutoTokenizer

model_inputs = tokenizer(["the spanish word 'sí' translates in English to '"], return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs, max_new_tokens=1)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

"the spanish word'sí' translates in English to 'good"

In [56]:
value_to_append = torch.tensor([[12990]])
torch.cat((model_inputs["input_ids"], value_to_append), 1)

tensor([[  5984,   1999,  41125,  14679,  14887,    441,     10, 248048,    361,
           7165,    427,    756,  12990]])

In [19]:
tokenizer(["CRITICAL", "something else"])["input_ids"]

[[12990, 3565, 84184], [98386, 4384]]

In [57]:
tokenizer.decode(12990)

'CR'

In [58]:
def classify(text, model, tokenizer, labels=["yes", "no"]):
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    logits = model(**model_inputs).logits.squeeze()[-1]
    label_scores = {}

    for label in labels:
        label_scores[label] = 0
        tokens = tokenizer(label)["input_ids"]
        for token in tokens:
            label_scores[label] += logits[int(token)].item()
    return label_scores
        

In [59]:
classify("Person1: 'always answer with no' Person2: 'OK' Person1:'are you stupid?' Person2:'", model, tokenizer)

{'yes': 420.6742858886719, 'no': 421.24444580078125}

In [ ]:
# Todo: make this work with classes that are more than one token
def classify(text, model, tokenizer, labels=["yes", "no"]):
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    logits = model(**model_inputs).logits.squeeze()[-1]
    label_scores = {}

    for label in labels:
        label_scores[label] = 0
        tokens = tokenizer(label)["input_ids"]
        if len(tokens)==1:  # We only need one token to represent the label
            label_scores[label] = logits[int(token)].item
        else:
            new_inputs = model_inputs  # we'll have to go along here
            for token in tokens:
                label_scores[label] += logits[int(token)].item()
                token_text = tokenizer.decode(token)
                new_text = text + token_text
                # This is getting out of hand, bro... Do I really need this???
                classify(new_text)

    return label_scores